### challenge url: https://www.hackerearth.com/challenges/competitive/lunar-rock-hackerearth-data-science-competition/machine-learning/lunar-rock-recognition-43274e07-04533c43/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install keras_efficientnets

In [0]:
import os
import shutil

In [4]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [0]:
from keras.layers import *
from keras.optimizers import *

In [6]:
!unzip drive/My\ Drive/DataSet

Archive:  drive/My Drive/DataSet.zip
   creating: DataSet/
 extracting: DataSet/Test Images.zip  
  inflating: DataSet/test.csv        
  inflating: DataSet/train.csv       
 extracting: DataSet/Train Images.zip  


In [0]:
# !unzip DataSet/Test\ Images.zip

In [9]:
len(os.listdir('Train Images/'))

2

In [0]:
train_df = pd.read_csv('DataSet/train.csv')
test_df = pd.read_csv('DataSet/test.csv')


In [17]:
train_df.shape

(11998, 2)

## Data Generator 

In [0]:
if not os.path.isdir('Test Images//test/test_data'):
  os.mkdir('Test Images/test/test_data')



In [0]:
# make subdirectory of each class
for each in os.listdir('Test Images/test/'):
  dest = 'Test Images/test/test_data'
  src = os.path.join('Test Images/test', each)
  shutil.move(src, dest)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_img_gen = ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 60,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
)

test_img_gen = ImageDataGenerator(
    rescale = 1/255.0,
)

In [68]:
train_gen = train_img_gen.flow_from_directory(
    'Train Images/',
    batch_size = 16,
)

Found 11998 images belonging to 2 classes.


In [69]:
test_gen = test_img_gen.flow_from_directory(
    'Test Images/test',
    batch_size = 16,
)

Found 7534 images belonging to 1 classes.


In [70]:
train_gen.class_indices

{'Large': 0, 'Small': 1}

### Classifier Model

In [50]:
!pip install keras_radam

  Created wheel for keras-radam: filename=keras_radam-0.15.0-cp36-none-any.whl size=14686 sha256=882f95991f7eb27dd4bad512a7d61612a5afadcdb2fa5781d7a9f3dde5c1fb0f
  Stored in directory: /root/.cache/pip/wheels/79/a0/c0/670b0a118e8f078539fafec7bd02eba0af921f745660c7f83f
Successfully built keras-radam


In [0]:
from keras_radam import RAdam

In [0]:
from keras.callbacks import Callback,  EarlyStopping

In [0]:
import keras_radam
import keras_efficientnets as eff

In [0]:
from keras.layers import *
from keras.optimizers import *
from keras.models import *

In [0]:
# early stopping
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.005, patience=3)

# models checkpoints-- saves best only 
cp_efficient = keras.callbacks.ModelCheckpoint('efficient_net_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_xception = keras.callbacks.ModelCheckpoint('xception_net_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_vgg = keras.callbacks.ModelCheckpoint('vgg.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_resnet = keras.callbacks.ModelCheckpoint('resnet.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_nas = keras.callbacks.ModelCheckpoint('nas_net.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)

In [0]:
efficient = eff.EfficientNetB5((256, 256, 3), classes=2, include_top=False, weights='imagenet')

In [83]:
### adding layers for flower classification

eff_avg = GlobalAveragePooling2D()(efficient.output)
eff_fc_1 = Dense(256, activation='relu')(eff_avg)
eff_fc_2 = Dense(128, activation='relu')(eff_fc_1)
eff_d = Dropout(0.25)(eff_fc_2)
eff_fc_3 = Dense(2, activation='softmax')(eff_d)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
es = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
cp_efficient = keras.callbacks.ModelCheckpoint('efficient_net_weights.h5', monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)

In [0]:
efficient = Model(efficient.input, outputs  = eff_fc_3)

In [0]:
radam = RAdam(learning_rate=0.000001)

In [0]:
efficient.compile(loss='categorical_crossentropy', optimizer=radam, metrics=['accuracy'])

In [91]:
efficient_hist = efficient.fit_generator(train_gen, steps_per_epoch=11998/16, epochs=5, callbacks=[es, cp_efficient])

Epoch 1/5
750/749 [==============================] - 1186s 2s/step - loss: 0.0203 - acc: 0.9931
Epoch 2/5
750/749 [==============================] - 1139s 2s/step - loss: 0.0195 - acc: 0.9942
Epoch 3/5
533/749 [====================>.........] - ETA: 5:29 - loss: 0.0195 - acc: 0.9938

KeyboardInterrupt: ignored

In [0]:
efficient.load_weights('efficient_net_weights.h5')

In [0]:
idx_to_class = dict((value, key) for key, value in train_gen.class_indices.items())

In [100]:
idx_to_class

{0: 'Large', 1: 'Small'}

In [0]:
if not os.path.isdir('Test Images/test'):
  os.mkdir('Test Images//test')



In [0]:
# make subdirectory of each class
for each in os.listdir('Test Images/'):
  dest = 'Test Images/test'
  src = os.path.join('Test Images/', each)
  shutil.move(src, dest)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [110]:
test_generator = test_datagen.flow_from_directory(
        'Test Images/test',
        shuffle = False,
        class_mode='categorical',
        batch_size=16
)

Found 7534 images belonging to 1 classes.


In [0]:
pred = efficient.predict_generator(test_generator, steps=7535/16)

In [0]:
pred = np.argmax(pred, axis = 1)

In [0]:
pred_ = [idx_to_class[each] for each in pred]

In [0]:
test_df.Class = pred_

In [121]:
test_df.head()

,Image_File,Class
0,lg 988 (1).png,Small
1,lg 988 (10).png,Small
2,lg 988 (100).png,Small
3,lg 988 (101).png,Small
4,lg 988 (102).png,Small


In [0]:
test_df.to_csv('submission.csv', index=False)

In [0]:
images = os.listdir('Test Images/test/test_data')

In [127]:
len(images)

7534

In [0]:
d = dict(zip(test_df.Image_File, test_df.Class))

In [0]:
for each in images:
  x = keras.preprocessing.image.load_img('Test Images/test/test_data/'+each, target_size=(256, 256))
  x_ = keras.preprocessing.image.img_to_array(x)
  x__ =np.expand_dims(x_, axis=0)
  d[each] = idx_to_class[np.argmax(efficient.predict(x__))]

In [0]:
x  = keras.preprocessing.image.load_img('Test Images/test/test_data/'+images[0], target_size=(256, 256))

In [0]:
x_ = keras.preprocessing.image.img_to_array(x)

In [160]:
test_df.head()

,Image_File,Class
0,lg 988 (1).png,Small
1,lg 988 (10).png,Small
2,lg 988 (100).png,Small
3,lg 988 (101).png,Small
4,lg 988 (102).png,Small


In [0]:
test_df_archive = test_df

In [0]:
test_df.Class = [d[each] for each in test_df.Image_File]

In [164]:
test_df.head()

,Image_File,Class
0,lg 988 (1).png,Large
1,lg 988 (10).png,Large
2,lg 988 (100).png,Large
3,lg 988 (101).png,Large
4,lg 988 (102).png,Large


In [0]:
x__ =np.expand_dims(x_, axis=0)

In [155]:
x__.shape

(1, 256, 256, 3)